# Relative GLAM Fit
This notebook runs the runs two versions of the GLAM: one that allows gamma to be a free parameter (BiasModel) and one with gamma held at 1 (NoBiasModel). It also runs a model comparision to determine which of the two variants is a better fit to the data

## Housekeeping
### Load Necessary Libraries

In [1]:
import warnings
from os.path import join

import glambox as gb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Ignore future warnings re: multi-indexing
warnings.simplefilter(action="ignore", category=FutureWarning)

# Fix for compilation error (as per Felix Molter)
# https://stackoverflow.com/a/51312739
import theano

theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"
theano.config.cxx


'/usr/bin/clang++'

### Set MCMC Sampling Parameters

In [2]:
# Set MCMC parameters
n_tune = 5000
n_draws = 5000
n_chains = 2

### Set Other Miscellaneous Variables

In [3]:
# Set error weight
error_weight = 0.01

# set random seed for replicability
np.random.seed(5)

# set monkey ID
monkeys = ["Monkey C", "Monkey K"]

### Set Data Grouping Flag
This notebook can be run treating each monkey as a subject, or each session as a subject. The data for the two versions are are in separate excel files. The only difference between the two is the input to the 'subject' field. 

**To select the correct input file, set the variable DATA_GROUPING to either 'session' or 'monkey'.** 'monkey' is the default option

In [4]:
DATA_GROUPING = "monkey"

In [5]:
data = pd.read_csv("GLAMdata_"+ DATA_GROUPING + ".csv")

# display the unique subject numbers to make sure you set the desired data grouping flag
print(data.subject.unique())

[0 1]


## Model Fitting

### Setup

In [6]:
# initialize GLAM model objects
BiasModel = gb.GLAM(data=data)
NoBiasModel = gb.GLAM(data=data)

In [7]:
# define models. 
# NOTE: This is where you specify that gamma=1 for the No Bias Model 
BiasModel.make_model(kind="individual", error_weight=error_weight)
NoBiasModel.make_model(kind="individual", gamma_val=1, error_weight=error_weight)

Generating single subject models for 2 subjects...
Elemwise{add,no_inplace}.0
Elemwise{add,no_inplace}.0
Elemwise{add,no_inplace}.0
Elemwise{add,no_inplace}.0
Elemwise{add,no_inplace}.0
Elemwise{add,no_inplace}.0
Generating single subject models for 2 subjects...
Elemwise{add,no_inplace}.0
Elemwise{add,no_inplace}.0
Elemwise{add,no_inplace}.0
Elemwise{add,no_inplace}.0
Elemwise{add,no_inplace}.0
Elemwise{add,no_inplace}.0


### Parameter estimation
#### Bias Model (free gamma)

In [8]:
BiasModel.fit(tune=n_tune, draws=n_draws, chains=n_chains)

Fitting 2 model(s) using MCMC...
  Fitting model 1 of 2...


Multiprocess sampling (2 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 2 chains: 100%|███████████████| 20000/20000 [05:06<00:00, 65.24draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 2 of 2...


Multiprocess sampling (2 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 2 chains: 100%|███████████████| 20000/20000 [04:28<00:00, 74.56draws/s]
The number of effective samples is smaller than 10% for some parameters.


/!\ Automatically setting parameter precision...


#### No Bias Model (gamma = 1) 

In [9]:
NoBiasModel.fit(tune=n_tune, draws=n_draws, chains=n_chains)

Fitting 2 model(s) using MCMC...
  Fitting model 1 of 2...


Multiprocess sampling (2 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [v]
Sampling 2 chains: 100%|██████████████| 20000/20000 [03:00<00:00, 110.69draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 2 of 2...


Multiprocess sampling (2 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [v]
Sampling 2 chains: 100%|██████████████| 20000/20000 [02:47<00:00, 119.21draws/s]
The number of effective samples is smaller than 10% for some parameters.


/!\ Automatically setting parameter precision...


### View Parameter Estimates

In [10]:
# display estimates for bias model
BiasModel.estimates

,gamma,gamma_hpd_2.5,gamma_hpd_97.5,s,s_hpd_2.5,s_hpd_97.5,subject,t0,t0_hpd_2.5,t0_hpd_97.5,tau,tau_hpd_2.5,tau_hpd_97.5,v,v_hpd_2.5,v_hpd_97.5
0,0.089,0.064114,0.115126,0.195,0.193093,0.197603,0,0.0,0.0,0.0,0.267,0.260717,0.272904,2.312,2.301724,2.318995
1,-0.061,-0.081234,-0.031275,0.171,0.169040,0.172939,1,0.0,0.0,0.0,0.306,0.298942,0.312676,2.204,2.194551,2.212125


In [11]:
# display estimates for no bias model
NoBiasModel.estimates

,gamma,gamma_hpd_2.5,gamma_hpd_97.5,s,s_hpd_2.5,s_hpd_97.5,subject,t0,t0_hpd_2.5,t0_hpd_97.5,tau,tau_hpd_2.5,tau_hpd_97.5,v,v_hpd_2.5,v_hpd_97.5
0,1.0,1.0,1.0,0.206,0.203994,0.208785,0,0.0,0.0,0.0,0.186,0.180509,0.190166,2.353,2.343694,2.361103
1,1.0,1.0,1.0,0.207,0.204411,0.209429,1,0.0,0.0,0.0,0.203,0.198640,0.208906,2.272,2.264483,2.282617


#### Create traceplots for fitted parameters
This is a quality assurance measure so that we can visually assess model convergence

In [ ]:
for i, monkey in enumerate(monkeys):
    fig, axs = gb.plots.traceplot(BiasModel.trace[i])
    fig.suptitle(monkey.capitalize(), y=1.02)
    
for i, monkey in enumerate(monkeys):
    fig, axs = gb.plots.traceplot(NoBiasModel.trace[i])
    fig.suptitle(monkey.capitalize(), y=1.02)
        

## Model Comparison
The following section assesses which of the two model variants is the better fit to the data. For each subject, the listed dWAIC will be 0 for the better fitting model and >0 for the other. In the main text, as was done in Thomas et al., 2019, the dWAIC value is signed and based on subtracting the NoBiasModel from the BiasModel. Thus, a negative dWAIC is indicative of the BiasModel being the better fit.

In [ ]:
# run model comparison function
modelComp = gb.analysis.compare_models(models=[BiasModel, NoBiasModel])

# add model variant label
# model 0 is the BiasModel and model 1 is the NoBias Model 
# (based on the input order to the function) 
modelComp['model'].loc[(modelComp['model']==0)]= 'BiasModel'
modelComp['model'].loc[(modelComp['model']==1)]= 'NoBiasModel'

In [15]:
if DATA_GROUPING == "session":
    # data for 'subject' 29-53 belong to Monkey K
    # data for 'subject' 0-28 belong to Monkey C
    monkeyKIdx = modelComp['subject'] >= 29
    modelComp.insert(0,'id',monkeyKIdx)
    modelComp['id'].loc[(modelComp['id']==True)]= monkeys[1]
    modelComp['id'].loc[(modelComp['id']==False)]= monkeys[0]
else:
    modelComp['subject'].loc[(modelComp['subject']==0)]= monkeys[0]
    modelComp['subject'].loc[(modelComp['subject']==1)]= monkeys[1]
    
modelComp

,subject,model,WAIC,pWAIC,dWAIC,weight,SE,dSE,var_warn
0,Monkey C,BiasModel,-15396.9,4.37,0,0.92,246.87,0,0
1,Monkey C,NoBiasModel,-12039,3.22,3357.87,0.08,247.02,139.3,0
2,Monkey K,BiasModel,-18570.9,4.22,0,1,232.64,0,0
3,Monkey K,NoBiasModel,-10411.2,3.03,8159.71,0,224.5,177.05,0
